# Phase transition detection in 2-dimensional Ising model

Using Neural Network, we will detect the critical temperature $\beta_{cr} = 0.440686$.

In [22]:
import numpy as np
import tensorflow as tf
import keras
import pylab as plt
from numpy.linalg import solve

In [ ]:
# number of configuration
!ls -l conf|grep npy|wc -l

In [24]:
# list of parameters
prm_list = [
            # beta, file_name, 
             [0.90,"conf/L32b090_"],
             [0.85,"conf/L32b085_"],
            
             [0.80,"conf/L32b080_"],
             [0.70,"conf/L32b070_"],
            
             [0.65,"conf/L32b065_"],
             [0.60,"conf/L32b060_"],
            
             [0.55,"conf/L32b055_"],
             [0.50,"conf/L32b050_"],
            
             [0.47,"conf/L32b047_"],
             [0.42,"conf/L32b042_"],
            
             [0.40,"conf/L32b040_"],
             [0.35,"conf/L32b035_"],
            
             [0.30,"conf/L32b030_"],
             [0.25,"conf/L32b025_"],
            
             [0.20,"conf/L32b020_"],
             [0.15,"conf/L32b015_"],
            
             [0.10,"conf/L32b010_"],
             [0.05,"conf/L32b005_"],
            
             [0.00,"conf/L32b000_"]
]
# beta19 x conf100 = 1900 data

In [ ]:
print("Reading training data")
for iconf in range(2):
  print("beta=090")
  file = f"conf/L32b090_{iconf}.npy"
  sc = np.load(file)
  plt.imshow(sc, cmap='gray')
  plt.show()
for iconf in range(2):
  print("beta=040")
  file = f"conf/L32b040_{iconf}.npy"
  sc = np.load(file)
  plt.imshow(sc, cmap='gray')
  plt.show()
for iconf in range(2):
  print("beta=000")
  file = f"conf/L32b000_{iconf}.npy"
  sc = np.load(file)
  plt.imshow(sc, cmap='gray')
  plt.show()

# Feature labeling and data separation

In [26]:
nconf = 100 # The number of configurations per beta
betacr = 0.440686 # critical temperature for 2d Ising model
#
data = []
labels = []
betas = []
nprm=len(prm_list)
for ibeta in range(nprm):
  beta = prm_list[ibeta][0]
  fname = prm_list[ibeta][1]
  for itrj in range(nconf):
    npsc = np.load(f"{fname}{itrj}.npy")
    data.append(npsc)
    if beta > betacr:
      labels.append([0,1])
    else:
      labels.append([1,0])
    betas.append(beta)
data = np.array(data)
labels = np.array(labels)
#
train_data=data[0::2]
train_labels=labels[0::2]
train_betas=betas[0::2] # this will not be used in training
#
val_data=data[1::2]
val_labels=labels[1::2]
val_betas=betas[1::2]
#
# beta18 x conf100 = 1800 data
# beta18 x conf50 = 900 training_data(18x50,same beta = 50) + 900 val_data

In [ ]:
print("train_data.shape = ", train_data.shape)
print("val_data.shape = ", val_data.shape)

## Model Definition

In [28]:
tf.random.set_seed(12345)
model_FC = keras.Sequential([
    keras.layers.Flatten(input_shape=(32, 32)),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(2, activation='softmax')
])

In [ ]:
model_FC.summary()

In [43]:
model_FC.compile(optimizer = keras.optimizers.Adam(learning_rate = 1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model_FC.fit(train_data, train_labels, epochs=100)

In [ ]:
xs=[]
y1s=[]
y2s=[]
Ndatamax = 950
Nsameclass = 50
for ii in range(0,Ndatamax,Nsameclass):
  res = model_FC(val_data[ii:ii+Nsameclass])
  x = val_betas[ii]
  y1= np.mean(res.numpy().T[0] )
  y2=np.mean(res.numpy().T[1] )
  xs.append( x)
  y1s.append( y1 )
  y2s.append( y2 )
  print(x,y1,y2)
plt.axvline(x=0.440686, ymin=0, ymax=1, ls="dashed",color="gray",label="Critical(Ans)")
plt.plot(xs,y1s,label="class:High",marker="o",color="red")
plt.plot(xs,y2s,label="class:Low",marker="o",color="blue")
plt.legend()
plt.xlabel(r"$\beta=1/T$")
plt.ylabel(r"Prob")
plt.show()

In [ ]:
xs=[]
y1s=[]
y2s=[]
Ndatamax = 950
Nsameclass = 50
for ii in range(0,Ndatamax,Nsameclass):
  res = model_FC(val_data[ii:ii+Nsameclass])
  x = val_betas[ii]
  y1= np.mean(res.numpy().T[0] )
  y2=np.mean(res.numpy().T[1] )
  xs.append( x)
  y1s.append( y1 )
  y2s.append( y2 )
  print(x,y1,y2)
plt.axvline(x=0.440686, ymin=0, ymax=1, ls="dashed",color="gray",label="Critical(Ans)")
plt.plot(xs,y1s,label="class:High",marker="o",color="red")
plt.plot(xs,y2s,label="class:Low",marker="o",color="blue")
plt.legend()
plt.xlabel(r"$\beta=1/T$")
plt.ylabel(r"Prob")
plt.xlim(0.44, 0.45)
plt.ylim(0.45, 0.55)
plt.show()

In [ ]:
# Calculating intersection
u1,v1 = xs[9],y1s[9]
u2,v2 = xs[8],y1s[8]
w1,t1 = xs[9],y2s[9]
w2,t2 = xs[8],y2s[8]
print(u1,v1)
print(u2,v2)
#
print(w1,t1)
print(w2,t2)

tan1=(v2-v1)/(u2-u1)
tan2=(t2-t1)/(w2-w1)
MatA = [[1, -tan1],
        [1, -tan2]]
 
vecB = [v1-tan1*u1,
        t1-tan2*w1]
#
sol = solve(MatA, vecB)
print("sol x,y = ", sol)
#
xx = np.linspace(0,1)
yy = tan1*(xx-u1)+v1
plt.plot(xx,yy)
#
yy = tan2*(xx-w1)+t1
plt.plot(xx, yy)
plt.ylim(0,1)
plt.show()
#
beta_cr = 0.440686
er = round(abs(beta_cr - sol[1])/beta_cr  *100,2)
print(f"Relative error = {er} %")

# Convolutional neural network

CNN

In [48]:
tf.random.set_seed(12345)
model_CNN = keras.Sequential([
    keras.layers.Conv2D(filters =  1,
                        kernel_size=(4, 4), 
                        activation='relu', 
                        input_shape=(32, 32, 1) 
                        ),
    keras.layers.Flatten(),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(2, activation='softmax')
])

In [ ]:
model_CNN.summary()

In [73]:
model_CNN.compile(optimizer = keras.optimizers.Adam(learning_rate = 1e-3), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
train_data_cnn=np.array(train_data)
train_data_cnn = train_data_cnn.reshape(train_data.shape[0], train_data.shape[1], train_data.shape[2], 1)
train_data_cnn.shape

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(filepath = "model_at_epoch_{epoch}.keras"),
    keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 10),
]

model_CNN.fit(
    train_data_cnn,
    train_labels,
    epochs = 1000,
    validation_split = 0.15,
    callbacks = callbacks,
)

In [ ]:
xs=[]
y1s=[]
y2s=[]
Ndatamax = 950
Nsameclass = 50
for ii in range(0,Ndatamax,Nsameclass):
  res = model_CNN(val_data[ii:ii+Nsameclass])
  x = val_betas[ii]
  y1= np.mean(res.numpy().T[0] )
  y2=np.mean(res.numpy().T[1] )
  xs.append( x )
  y1s.append( y1 )
  y2s.append( y2 )
  print(x,y1,y2)
plt.axvline(x=0.440686, ymin=0, ymax=1, ls="dashed",color="gray",label="Critical")
plt.plot(xs,y1s,label="class:High",marker="o",color="red")
plt.plot(xs,y2s,label="class:Low",marker="o",color="blue")
plt.legend()
plt.xlabel(r"$\beta=1/T$")
plt.ylabel(r"Prob")
plt.show()

In [ ]:
plt.axvline(x=0.440686, ymin=0, ymax=1, ls="dashed",color="gray",label="Critical")
plt.plot(xs,y1s,label="class:High",marker="o",color="red")
plt.plot(xs,y2s,label="class:Low",marker="o",color="blue")
plt.legend()
plt.xlabel(r"$\beta=1/T$")
plt.ylabel(r"Prob")
plt.xlim(0.44, 0.45)
plt.ylim(0.45, 0.55)
plt.show()

In [ ]:
u1,v1 = xs[9],y1s[9]
u2,v2 = xs[8],y1s[8]
w1,t1 = xs[9],y2s[9]
w2,t2 = xs[8],y2s[8]
print(u1,v1)
print(u2,v2)
#
print(w1,t1)
print(w2,t2)

from numpy.linalg import solve

tan1=(v2-v1)/(u2-u1)
tan2=(t2-t1)/(w2-w1)
MatA = [[1, -tan1],
        [1, -tan2]]
 
vecB = [v1-tan1*u1,
        t1-tan2*w1]
#
sol = solve(MatA, vecB)
print("sol x,y = ", sol)
#
xx = np.linspace(0,1)
yy = tan1*(xx-u1)+v1
plt.plot(xx,yy)
#
yy = tan2*(xx-w1)+t1
plt.plot(xx, yy)
plt.ylim(0,1)
plt.show()
#
beta_cr = 0.440686
er = round(abs(beta_cr - sol[1])/beta_cr  *100,2)
print(f"Relative error = {er} %")